# Spectral Analysis

Moments, Spectral line fitting, Continuum subtraction etc.



##  Continuum Subtraction on an Image Cube (imcontsub)

One method to separate line and continuum emission in an image cube is
to specify a number of line-free channels in that cube, make a linear
fit to the visibilities in those channels, and subtract the fit from the
whole cube. Note that the task **uvcontsub** serves a similar purpose
but the subtraction is performed in visibility space (see [UV Continuum
Subtraction](resolveuid/2c85a22fd1004fb194cf1890672ad94a)). The
**imcontsub** task will perform&nbsp;a polynomial baseline fit to the
specified channels from an image cube and subtract it from all
channels.  
The default inputs are:

\#&nbsp; imcontsub :: Continuum subtraction on images  
imagename&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp; \#&nbsp;
Name of the input image  
linefile&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp; Output line image file name  
contfile&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp; Output continuum image file name  
fitorder&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 0&nbsp;&nbsp;
\#&nbsp; Polynomial order for the continuum estimation  
region&nbsp;&nbsp;&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp; \#&nbsp; Image region or name to process see viewer  
box&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp; \#&nbsp; Select one or
more box regions  
chans&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp; \#&nbsp; Select the channel(spectral) range  
stokes&nbsp;&nbsp;&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp; \#&nbsp; Stokes params to image (I,IV,IQU,IQUV)

Area selection using *box* and *region* is detailed in the
[box](#region-selection--box-) and [region](#regions--region-) sections.
Image cube plane selection using *chans* and *stokes* are described in
the [Plane selection](#plane-selection--chans--stokes-) section.

**ALERT:** **imcontsub** has issues when the image does not contain a
spectral or stokes axis. Errors are generated when run on an image
missing one or both of these axes. You will need to use the toolkit
(e.g. the **ia.adddegaxes** method) to add degenerate missing axes to
the image.



###  Examples for **imcontsub**

For example, we first make a clean image from data in which&nbsp;no
uv-plane continuum subtraction has been performed:

\

## Now clean, keeping all the channels except first and last  
default('clean')  
vis = 'ngc5921.demo.src.split.ms'  
imagename = 'ngc5921.demo.nouvcontsub'  
mode = 'channel'  
nchan = 61  
start = 1  
width = 1  
imsize = \[256,256\]  
psfmode = 'clark'  
imagermode = ''  
cell = \[15.,15.\]  
niter = 6000  
threshold='8.0mJy'  
weighting = 'briggs'  
robust = 0.5  
mask = \[108,108,148,148\]  
interactive=False  
clean()  
  
\

## It will have made the image:  
\

## -----------------------------  
\

## ngc5921.demo.nouvcontsub.image  
  
\

## You can view this image  
viewer('ngc5921.demo.nouvcontsub.image')

Channels 0 through 4 and 50 through 60 are line-free.&nbsp;Continuum
subtraction is then performed with:

default('imcontsub')  
imagename = 'ngc5921.demo.nouvcontsub.image'  
linefile&nbsp; = 'ngc5921.demo.nouvcontsub.lineimage'  
contfile&nbsp; = 'ngc5921.demo.nouvcontsub.contimage'  
fitorder&nbsp; = 1  
chans&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = '0\~4,50\~60'  
stokes&nbsp;&nbsp;&nbsp; = 'I'  
imcontsub()

&nbsp;



## Computing the Moments of an Image Cube (**immoments**)

For spectral line datasets, the output of the imaging process is an
image cube, with a frequency or velocity channel axis in addition to the
two sky coordinate axes. This can be most easily thought of as a series
of image planes stacked along the spectral dimension. A useful product
to compute is to collapse the cube into a moment image by taking a
linear combination of the individual planes:

$M_m(x_i,y_i) = \sum_k^N w_m(x_i,y_i,v_k)\,I(x_i,y_i,v_k)$

for pixel i and channel k in the cube I. There are a number of choices
to form the moment-m, usually approximating some polynomial expansion of
the intensity distribution over velocity mean or sum, gradient,
dispersion, skew, kurtosis, etc. There are other possibilities (other
than a weighted sum) for calculating the image, such as median
filtering, finding minima or maxima along the spectral axis, or absolute
mean deviations. And the axis along which to do these calculations need
not be the spectral axis (i.e. do moments along Dec for a RA-Velocity
image). We will treat all of these as generalized instances of a
&ldquo;moment&rdquo; map.  
  
The **immoments** task will compute basic moment images from a cube. The
default inputs are:

\#&nbsp; immoments :: Compute moments of an image cube:  
imagename&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp;&nbsp; Input image name  
moments&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \[0\]&nbsp;&nbsp; \#&nbsp;
List of moments you would like to compute  
axis&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; =
'spectral'&nbsp;&nbsp; \#&nbsp; The moment axis: ra, dec, lat, long,
spectral, or stokes  
region&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp; Image Region.&nbsp; Use viewer  
box&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp; Select one or more box regions  
chans&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp; Select the channel(spectral) range  
stokes&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp; Stokes params to image (I,IV,IQU,IQUV)  
mask&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp; mask used for selecting the area of the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image to calculate the moments on  
includepix&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp; \#&nbsp; Range of pixel values to include  
excludepix&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp; \#&nbsp; Range of pixel values to exclude  
outfile&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ''&nbsp;&nbsp;
\#&nbsp; Output image file name (or root for multiple moments)

This task will operate on the input file given by *imagename* and
produce a new image or set of images based on the name given in
*outfile*.  
  
The *moments* parameter chooses which moments are calculated. The
choices for the operation mode are:

-   moments=-1&nbsp; - mean value of the spectrum
-   moments=0&nbsp;&nbsp; - integrated value of the spectrum
-   moments=1&nbsp;&nbsp; - intensity weighted coordinate; traditionally
    used to get 'velocity fields'
-   moments=2&nbsp;&nbsp; - intensity weighted dispersion of the
    coordinate; traditionally used to get 'velocity dispersion'
-   moments=3&nbsp;&nbsp; - median of I
-   moments=4&nbsp;&nbsp; - median coordinate
-   moments=5&nbsp;&nbsp; - standard deviation about the mean of the
    spectrum
-   moments=6&nbsp;&nbsp; - root mean square of the spectrum
-   moments=7&nbsp;&nbsp; - absolute mean deviation of the spectrum
-   moments=8&nbsp;&nbsp; - maximum value of the spectrum
-   moments=9&nbsp;&nbsp; - coordinate of the maximum value of the
    spectrum
-   moments=10&nbsp; - minimum value of the spectrum
-   moments=11&nbsp; - coordinate of the minimum value of the spectrum

The meaning of these is described in the [CASA Toolkit
Manual](https://casa.nrao.edu/docs/CasaRef/CasaRef.html), that describes
the associated
[image.moments](https://casa.nrao.edu/docs/CasaRef/image.moments.html#x62-620001.1.1)
tool.  
  
The *axis* parameter sets the axis along which the moment is
&ldquo;collapsed&rdquo; or calculated. Choices are: 'ra', 'dec', 'lat',
'long', 'spectral', or 'stokes'. A standard moment-0 or moment-1 image
of a spectral cube would use the default choice &rsquo;spectral&rsquo;.
One could make a position-velocity map by setting 'ra' or 'dec'.  
  
The *includepix* and *excludepix* parameters are used to set ranges for
the inclusion and exclusion of pixels based on values. For example,
i*ncludepix=\[0.05,100.0\]* will include pixels with values from 50 mJy
to 1000 Jy, and *excludepix=\[100.0,1000.0\]* will exclude pixels with
values from 100 to 1000 Jy.  
  
If a single moment is chosen, the outfile specifies the exact name of
the output image. If multiple moments are chosen, then outfile will be
used as the root of the output filenames, which will get different
suffixes for each moment. For image cubes that contain different beam
sizes for each plane, **immoments** will smooth all planes to the
largest beam size first, then collapse to the desired moment.

&nbsp;



### Hints for using **immoments**

In order to make an unbiased moment-0 image, do not put in any
thresholding using *includepix* or *excludepix*. This is so that the
(presumably) zero-mean noise fluctuations in off-line parts of the image
cube will cancel out. If you image has large biases, like a pronounced
clean bowl due to missing large-scale flux, then your moment-0 image
will be biased also. It will be difficult to alleviate this with a
threshold, but you can try.

To make a usable moment-1 (or higher) image, on the other hand, it is
critical to set a reasonable threshold to exclude noise from being added
to the moment maps. Something like a few times the rms noise level in
the usable planes seems to work (put into *includepix* or *excludepix*
as needed). Also use *chans* to ignore channels with bad data.

&nbsp;



### Examples using **immoments**

Below is an example for **immoments**:

default('immoments')  
imagename = 'ngc5921.demo.cleanimg'  
\

## Do first and second spectral moments  
axis&nbsp; = 'spectral'  
chans = ''  
moments = \[0,1\]  
\

## Need to mask out noisy pixels, currently done  
\

## using hard global limits  
excludepix = \[-100,0.009\]  
outfile = 'ngc5921.demo.moments'  
&nbsp;  
immoments()  
  
\

## It will have made the images:  
\

## --------------------------------------  
\

## ngc5921.demo.moments.integrated  
\

## ngc5921.demo.moments.weighted_coord

Other examples of NGC2403 (a moment-0 image of a VLA line dataset) and
NGC4826 (a moment-1 image of a BIMA CO line dataset) are shown in the
Figure
[below](http://casa.nrao.edu/casadocs/stable/image-analysis/dealing-with-images#figid-analysisfigmoments).

![https://raw.githubusercontent.com/0d5fd0d2ee99bce18c9c5fcce6f3439c8f9042ce.png](https://raw.githubusercontent.com/0d5fd0d2ee99bce18c9c5fcce6f3439c8f9042ce.png)

![https://raw.githubusercontent.com/4e1cdd67ee0996b1a1e19225e04cb383489aabd1.png](https://raw.githubusercontent.com/4e1cdd67ee0996b1a1e19225e04cb383489aabd1.png)

|         |                                                                                                   |
|---------|---------------------------------------------------------------------------------------------------|
| Type    | Figure                                                                                            |
| ID      | analysis-fig-moments                                                                              |
| Caption | NGC2403 VLA moment zero (left) and NGC4826 BIMA moment one (right) images as shown in the viewer. |



##  Generating Position-Velocity Diagrams (impv)

CASA can generate position-velocity (PV) diagrams via the task **impv**
or directly in the **viewer** (see [Image Cube
Visualization](https://casa.nrao.edu/casadocs-devel/stable/imaging/image-cube-visualization)**)**.&nbsp;
The viewer application calls the task:

\#&nbsp; impv :: Construct a position-velocity image by choosing two
points in the direction plane.  
imagename&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of the input
image  
outfile&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output image name.
If empty, no image is written.  
mode&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp; 'coords'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp; If 'coords', use start and end values. If 'length', use  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; center, length, and pa values.  
width&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Width of slice for
averaging pixels perpendicular to the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; slice. Must be an odd positive integer or valid  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; quantity. See help for details.  
unit&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp; 'arcsec'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp; Unit for the offset axis in the resulting image. Must be  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; a unit of angular measure.  
chans&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Channels to
use.&nbsp;  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; Channels must be contiguous. Default is to use all  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; channels.  
&nbsp;&nbsp;&nbsp;&nbsp;
region&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Region selection.
Default is entire image. No selection  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; is permitted in the direction plane.&nbsp;  
  
stokes&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
'I'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Stokes planes to
use. Planes must be contiguous. Default  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; is to use all stokes.  
mask&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Mask to use.
Default is none.  
&nbsp;&nbsp;&nbsp;&nbsp;
stretch&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
False&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Stretch the
mask if necessary and possible? Default False

PV diagrams are generated by &ldquo;slicing&rdquo; a datacube through
the RA/DEC planes. The &ldquo;slit&rdquo; can be defined either by
start/end coordinates or by a length, center coordinate, and position
angle. Averaged over the width of the &rsquo;slit&rsquo; the image cube
values are then stored in a new image with position and velocity as the
two axes. The slit position is specified by a start and end pixel in the
RA/DEC plane of the data cube. An angular unit can be set to define what
is stored in the resulting PV&nbsp;image.

&nbsp;



## 1-dimensional Smoothing (specsmooth)

To gain higher signal-to-noise of data cubes, one can smooth the data
along one dimension (for 2-dimensional smoothing, see **imsmooth**
[below](#2-dimensional-smoothing--image-convolution--imsmooth-)**)**.
Typically this is the spectral axis. Hanning and Boxcar smoothing
kernels are available in the task **specsmooth**:

\#&nbsp; specsmooth :: Smooth an image region in one dimension  
imagename&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of the input
image  
outfile&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output image
name.  
region&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Region selection.
Default is to use the full  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image.  
&nbsp;&nbsp;&nbsp;&nbsp;
box&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Rectangular region
to select in  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; direction plane. Default is to use the entire  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; direction plane.  
  
mask&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Mask to use.
Default is none..  
axis&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; The profile axis.
Default: use the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; spectral axis if one exists, axis 0  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; otherwise (\<0).  
function&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp; 'hanning'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp;
Convolution function. hanning and boxcar  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; are supported functions. Minimum match  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; is supported.  
dmethod&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;
'copy'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Decimation
method. '' means no  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; decimation, 'copy' and 'mean' are also  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; supported (minimum match).

The parameter *dmethod='copy'* allows one to only keep every nth
channel, if the smoothing kernel has a width of n. Leaving this
parameter empty will return the same size cube as the input and setting
it to &rsquo;mean&rsquo; will average planes using the kernel width.

&nbsp;

&nbsp;



##  &nbsp; Spectral Line fitting with **specfit**

**specfit** is a powerful task to perform spectral line fits in data
cubes. Three types of fitting functions are currently supported,
polynomials, Gaussians, and Lorentzians. **specfit** can fit these
functions in two ways: over data that were averaged across a region
(*multifit=False*) or on a pixel by pixel basis (*multifit=True*).

\#&nbsp; specfit :: Fit 1-dimensional Gaussians and/or polynomial models
to an image or image region  
imagename&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of the input
image  
box&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Rectangular box in
direction coordinate  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; blc, trc. Default: entire image ('').  
region&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Region of
interest. Default: Do  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; not use a region.  
chans&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Channels to use.
Channels must be  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; contiguous. Default: all channels ('').  
stokes&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Stokes planes to
use. Planes must be  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; contiguous. Default: all stokes ('').  
axis&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; The profile axis.
Default: use the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; spectral axis if one exists, axis 0  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; otherwise (\<0).  
mask&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Mask to use.
Default is  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; none..  
poly&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Order of
polynomial element.&nbsp; Default: do  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; not fit a polynomial (\<0).  
estimates&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of file
containing initial estimates.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; Default: No initial estimates ('').  
&nbsp;&nbsp;&nbsp;&nbsp;
ngauss&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Number of Gaussian
elements.&nbsp; Default: 1.  
&nbsp;&nbsp;&nbsp;&nbsp;
pampest&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Initial estimate
of PCF profile (gaussian  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; or lorentzian) amplitudes.  
&nbsp;&nbsp;&nbsp;&nbsp; pcenterest&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Initial estimate
PCF profile centers, in  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; pixels.  
&nbsp;&nbsp;&nbsp;&nbsp; pfwhmest&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Initial estimate
PCF profile FWHMs, in  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; pixels.  
&nbsp;&nbsp;&nbsp;&nbsp;
pfix&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; PCF profile
parameters to fix during fit.  
&nbsp;&nbsp;&nbsp;&nbsp;
pfunc&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; PCF singlet
functions to fit. 'gaussian'  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; or 'lorentzian' (minimal match  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; supported). Unspecified means all  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; gaussians.  
  
minpts&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Minimum number of
unmasked points  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; necessary to attempt fit.  
multifit&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; If true, fit a
profile along the desired  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; axis at each pixel in the specified  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; region. If false, average the non-fit  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; axis pixels and do a single fit to that  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; average profile. Default False.  
&nbsp;&nbsp;&nbsp;&nbsp;
amp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of amplitude
solution image. Default:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; do not write the image ('').  
&nbsp;&nbsp;&nbsp;&nbsp;
amperr&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of amplitude
solution error image.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; Default: do not write the image ('').  
&nbsp;&nbsp;&nbsp;&nbsp;
center&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of center
solution image. Default: do  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; not write the image ('').  
&nbsp;&nbsp;&nbsp;&nbsp; centererr&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of center
solution error image.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; Default: do not write the image ('').  
&nbsp;&nbsp;&nbsp;&nbsp;
fwhm&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of fwhm
solution image. Default: do  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; not write the image ('').  
&nbsp;&nbsp;&nbsp;&nbsp;
fwhmerr&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of fwhm
solution error image.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; Default: do not write the image ('').  
&nbsp;&nbsp;&nbsp;&nbsp; integral&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Prefix of name of
integral solution image.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; Name of image will have gaussian  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; component number appended.&nbsp; Default: do  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; not write the image ('').  
&nbsp;&nbsp;&nbsp;&nbsp; integralerr&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Prefix of name of
integral error solution  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image. Name of image will have gaussian  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; component number appended.&nbsp; Default: do  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; not write the image ('').  
  
model&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of model
image. Default: do not write  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; the model image ('').  
residual&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of residual
image. Default: do not  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; write the residual image ('').  
wantreturn&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Should a record
summarizing the results be  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; returned?  
logresults&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output results
to logger?  
gmncomps&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Number of
components in each gaussian  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; multiplet to fit  
gmampcon&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; The amplitude
ratio constraints for non-  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; reference components to reference  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; component in gaussian multiplets.  
gmcentercon&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; The center offset
constraints (in pixels)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; for non-reference components to reference  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; component in gaussian multiplets.  
gmfwhmcon&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; The FWHM&nbsp;
ratio constraints for non-  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; reference components to reference  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; component in gaussian multiplets.  
gmampest&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[0.0\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Initial
estimate of individual gaussian  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; amplitudes in gaussian multiplets.  
gmcenterest&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[0.0\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Initial
estimate of individual gaussian  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; centers in gaussian multiplets, in  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; pixels.  
gmfwhmest&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[0.0\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Initial
estimate of individual gaussian  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; FWHMss in gaussian multiplets, in pixels.  
gmfix&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Parameters of
individual gaussians in  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; gaussian multiplets to fix during fit.  
logfile&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; File in which to
log results. Default is  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; not to write a logfile.  
goodamprange&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[0.0\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Acceptable
amplitude solution range. \[0.0\]  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; =\> all amplitude solutions are  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; acceptable.  
goodcenterrange&nbsp;&nbsp;&nbsp;&nbsp; =&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[0.0\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Acceptable
center solution range in pixels  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; relative to region start. \[0.0\] =\> all  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; center solutions are acceptable.  
goodfwhmrange&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[0.0\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Acceptable
FWHM solution range in pixels.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; \[0.0\] =\> all FWHM solutions are  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; acceptable.  
sigma&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Standard deviation
array or image name.



###  Polynomial Fits

Polynomials can be fit by specifying the polynomial order in *poly*.
Negative orders will not fit any polynomials.



###  Lorentzian and Gaussian Fits

Gaussian and Lorentzian fits are very similar, they both require
amplitude, center, and FWHM to be fully specified. All of the following
discussion is thus valid for both functions. The parameter *pfunc*
controls whether Gaussian or Lorentzian functions are to be used.
Default is all Gaussians. *pfunc=\['L', 'G', 'G', 'L'\]* would use
Lorentzian, Gaussian, Gaussian, and Lorentzian components in the order
they appear in the estimates file (see below).



####  One or more single Gaussian/Lorentzian

For Gaussian and Lorentzian fits, the task will allow multiple
components and **specfit** will try to find the best solution. The
parameter *space*, however, is usually not uniform and to avoid local
minima in the goodness-of-fit space, one can provide initial start
values for the fits. This can be done either through the parameters
*pampest*, *pcenterest*, and *pfwhmest* for the amplitudes, center, and
FWHM estimates in image coordinates. *pfix* can take parameters that
specify fixed fit values. Any combination of the characters '*p'*
(peak), '*c'* (center), and '*f'* (fwhm) are permitted, e.g. '*fc*' will
hold the fwhm and the center constant during the fit. Fixed parameters
will have no errors associated with them in the solution. Alternatively,
a file with initial values can be supplied by the *estimates* parameter
(one Gaussian/Lorentzian parameter set per line). The file has the
following format:

    [peak intensity], [center], [fwhm], [optional fixed parameter string]

The first three values are required and must be numerical values. The
peak intensity must be expressed in map units, while the center and fwhm
must be specified in pixels. The fourth value is optional and if
present, represents the parameter(s) that should be held constant during
the fit (see above).  
  
An example estimates file is:

    

## estimates file indicating that two Gaussians should be fit
    

## first guassian estimate, peak=40, center at pixel number 10.5, 
    

## fwhm = 5.8 pixels, all parameters allowed to vary during
    

## fit
    40, 10.5, 5.8
    

## second Gaussian, peak = 4, center at pixel number 90.2, 
    

## fwhm = 7.2 pixels, hold fwhm constant
    4, 90.2, 7.2, f
    

## end file

and the output of a typical execution, e.g.

specfit(imagename='IRC10216_HC3N.cube_r0.5.image',  
region='specfit.crtf', multifit=F, estimates='', ngauss=2)  
  
(&rsquo;specfit.crtf&rsquo; is a CASA regions file, see Section D)  
will be  
  
Fit&nbsp;&nbsp; :  
&nbsp;&nbsp;&nbsp;
RA&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
:&nbsp;&nbsp; 09:47:57.49  
&nbsp;&nbsp;&nbsp;
Dec&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; :&nbsp;&nbsp;
13.16.46.46  
&nbsp;&nbsp;&nbsp; Stokes&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; : I  
&nbsp;&nbsp;&nbsp; Pixel&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; :
\[146.002, 164.499, 0.000,&nbsp; \*\]  
&nbsp;&nbsp;&nbsp; Attempted&nbsp;&nbsp;&nbsp; : YES  
&nbsp;&nbsp;&nbsp; Converged&nbsp;&nbsp;&nbsp; : YES  
&nbsp;&nbsp;&nbsp; Iterations&nbsp;&nbsp; : 28  
&nbsp;&nbsp;&nbsp; Results for component 0:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Type&nbsp;&nbsp;&nbsp;&nbsp;
: GAUSSIAN  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Peak&nbsp;&nbsp;&nbsp;&nbsp;
: 5.76 +/- 0.45 mJy/beam  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Center&nbsp;&nbsp; : -15.96
+/- 0.32 km/s  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
40.78 +/- 0.31 pixel  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; FWHM&nbsp;&nbsp;&nbsp;&nbsp;
: 7.70 +/- 0.77 km/s  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
7.48 +/- 0.74 pixel  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Integral : 47.2 +/- 6.0
mJy/beam.km/s  
&nbsp;&nbsp;&nbsp; Results for component 1:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Type&nbsp;&nbsp;&nbsp;&nbsp;
: GAUSSIAN  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Peak&nbsp;&nbsp;&nbsp;&nbsp;
: 4.37 +/- 0.33 mJy/beam  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Center&nbsp;&nbsp; : -33.51
+/- 0.58 km/s  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
23.73 +/- 0.57 pixel  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; FWHM&nbsp;&nbsp;&nbsp;&nbsp;
: 15.1 +/- 1.5 km/s  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
14.7 +/- 1.5 pixel  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Integral : 70.2 +/- 8.8
mJy/beam.km/s

If *wantreturn=True* (the default value), the task returns a python
dictionary (here captured in a variable with the inventive name of
'*fitresults'*) :

fitresults=specfit(imagename='IRC10216_HC3N.cube_r0.5.image',
region='specfit.rgn', multifit=F,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; estimates='', ngauss=2)

The values can then be used by other python code for further processing.

&nbsp;



#### Gaussian Multiplets

It is possible to fit a number of Gaussians together, as multiplets with
restrictions. All restrictions are relative to a reference Gaussian (the
zero&rsquo;th component of each multiplet). *gncomps* specifies the
number of Gaussians for each multiplets, and, in fact, a number of these
multiplets can be fit simultaneously. *gncomps=\[2,4,3\]*, e.g. fits a
2-component Gaussian, a 4-component Gaussian, and a 3-component Gaussian
all at once. The initial parameter estimates can be specified with the
*gmampest*, *gmcenterest*, and *gmfwhmest* parameters and the estimates
are simply listed in the sequence of *gncomps*. E.g. if
*gncomps=\[2,4,3\]* is specified with multiplet G0 consisting of 2
Gaussians a, b, multiplet G1 of 4 Gaussians c, d, e, f, and multiplet G2
of three Gaussians g, h, i, the parameter list in *gm\*est* would be
like *gm\*est=\[a,b,c,d,e,f,g,h,i\]*.  
  
Restrictions can be specified via the *gmampcon* parameter for the
amplitude ratio (non-reference to reference), *gmcentercon* for the
offset in pixels (to a reference), and *gmfwhmcon* for the FWHM ratio
(non-reference to reference). A value of 0 will not constrain anything.
The reference is always the zero&rsquo;th component in each multiplet,
in our example, Gaussians a, c, and g. They cannot be constrained. So
*gmncomps=\[2, 4, 3\]*, *gmampcon= \[ 0 , 0.2, 0 , 0.1, 4.5, 0 \]*,
*gcentercon=\[24.2, 45.6, 92.7, 0 , -22.8, -33.5\],* and *gfwhmcon=' '*
would constrain Gaussians b relative to a with a 24.2 pixel offset,
Gaussian d to c with a amplitude ratio of 0.2 and a 45.6 pixel offset,
Gaussian e to c with a offset of 92.7 pixels, etc. Restrictions will
overrule any estimates.  
  
The parameters *goodamprange*, *goodcenterrange*, and *goodfwhmrange*
can be used to limit the range of amplitude, center, and fwhm solutions
for all Gaussians.



####  Pixel-by-pixel fits

As mentioned above, **specfit** can also fit spectral cubes on a pixel
by pixel basis. In this case, one can choose to write none, any or all
of the solution and error images for Gaussian/Lorentzian fits via the
parameters *amp*, *amperr*, *center*, *centererr*, *fwhm*, and
*fwhmerr*. Each Gaussian component will produce a set of images *\_0,
\_1*, etc. suffixes. Writing analogous images for polynomial
coefficients is not yet supported although polynomial fits when
*multifit=True* is supported. Best fit coefficients are written to the
logger. Pixels for which fits were not attempted or did not converge
will be masked as bad.



##  Spatial Spectral Line Properties (**specflux**)

**specflux** calculates the flux as a function of frequency and velocity
over a selected spatial region. Flux densities of Jy/beam are being
converted to Jy by properly integrating over the selected region.  
  
The input parameters of **specflux** are:

\#&nbsp; specflux :: Report details of an image spectrum.  
imagename&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of the input
image  
box&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Rectangular region
to select in  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; direction plane. Default is to use the entire  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; direction plane.  
&nbsp;&nbsp;&nbsp;&nbsp;
region&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Region selection.
&nbsp;Default is to use the full  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image.  
  
chans&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Channels to use.
&nbsp;Default is to use all  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; channels.  
stokes&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Stokes planes to
use. &nbsp;Default is to  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; use all Stokes planes.  
mask&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Mask to use.
&nbsp;Default  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; is none.  
unit&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;
'km/s'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Unit to use
for the abscissa. Must be  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; conformant with a typical spectral axis  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; unit.  
major&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Major axis of
overriding restoring beam.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; If specified, must be a valid quantity.  
minor&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Minor axis of
overriding restoring beam.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; If specified, must be a valid quantity  
logfile&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; File which to
write details. Default is  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; to not write to a file.

The results can be written into a logfile to be plotted in other
packages.

&nbsp;



## Plot Spectra on a Map (plotprofilemap)

The **profilemap** task enables plotting spectra according to their
pointing directions (a.k.a. a profile map) in plots. The input should be
CASA image,or FITS format cube. Spectra within the cube are averaged
into a bin number specified with the *numpanels* keyword. Absent or
masked data are treated according to *plotmasked* keyword setting.

plotprofilemap(imagename='interesting_spectralcube_casaimge.im',  
figfile = 'grid_map.png',  
separatepanel=F,  
spectralaxis = 'velocity',  
title = 'myprofilemap',  
transparent = F,  
showaxislabel = True,  
showtick = True,  
showticklabel = True,  
pol=0,  
numpanels='8')

&nbsp;



## Calculation of Rotation Measures (**rmfit**)

**rmfit** is an image domain task that accepts an input cube image
containing Stokes Q and U axes and will generate the rotation measure by
performing a least square fit in the image domain to obtain the best fit
to the equation &nbsp;$\chi = \chi_0 + RM\times \lambda^2$.

The inputs to **rmfit** are:

\#&nbsp; rmfit :: Calculate rotation measure.  
imagename&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name(s) of the
input image(s). Must be specified.  
rm&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output rotation
measure image name. If not specified, no  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image is written.  
rmerr&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output rotation
measure error image name. If not  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; specified, no image is written.  
pa0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output position
angle (degrees) at zero wavelength image  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; name. If not specified, no image is written.  
pa0err&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output position
angle (degrees) at zero wavelength error  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image name. If not specified, no image is written.  
nturns&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output number of
turns image name. If not specified, no  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image is written.  
chisq&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output reduced chi
squared image name. If not specified,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; no image is written.  
sigma&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Estimate of the
thermal noise.&nbsp; A value less than 0 means  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; auto estimate.  
rmfg&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
0.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Foreground
rotation measure in rad/m/m to subtract.  
rmmax&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
0.0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Maximum rotation
measure in rad/m/m for which to solve.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; IMPORTANT TO SPECIFY.  
maxpaerr&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
1e+30&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Maximum input
position angle error in degrees to allow in  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; solution determination.

This task generates the rotation measure image from stokes Q and U
measurements at several different frequencies. You are required to
specify the name of at least one image with a polarization axis
containing stokes Q and U planes and with a frequency axis containing
more than two pixels. The frequencies do not have to be equally spaced
(i.e. the frequency coordinate can be a tabular coordinate). It will
work out the position angle images for you. You may also specify
multiple image names, in which case these images will first be
concatenated along the spectral axis using **ia.imageconcat**. The
requirements are that for all images, the axis order must be the same
and the number of pixels along each axis must be identical, except for
the spectral axis which may differ in length between images. The
spectral axis need not be contiguous from one image to another. See also
the i**magepol.fourierrotationmeasure** function for a new Fourier-based
approach.  
  
Rotation measure algorithms that work robustly are few. The main problem
is in trying to account for the $n- \pi$ ambiguity (see Leahy et al.1986
- Appendix A.1) <a href="#cit1" id="ref-cit1" class="ref-cit">[1]</a>
and the [MIRIAD
manual](http://www.cfa.harvard.edu/sma/miriad/manuals/SMAuguide/smauserhtml/imrm.html).

|                 |                                                                                                                                  |
|:----------------|----------------------------------------------------------------------------------------------------------------------------------|
| Citation Number | 1                                                                                                                                |
| Citation Text   | Leahy, J.\~P., Pooley, G.\~G., & Jagers, W.\~J. 1986, A&A, 156, 234 ([ADS](http://adsabs.harvard.edu/abs/1986A%26A...156..234L)) |

But as in all these algorithms, the basic process is that for each
spatial pixel, the position angle vs frequency data is fit to determine
the rotation measure and the position angle at zero wavelength (and
associated errors). An image containing the number of $n- \pi$ turns
that were added to the data at each spatial pixel and for which the best
fit was found can be written. The reduced $\chi^2$ image for the fits can also be
written. Any combination of output images can be written.

**NOTE**: No assessment of curvature (i.e. deviation from the simple
linear position angle - $\lambda^2$ functional form) is made.

The parameter *sigma* gives the thermal noise in Stokes Q and U. By
default it is determined automatically using the image data. But if it
proves to be inaccurate (maybe not many signal-free pixels), it may be
specified. This is used for calculating the error in the position angles
(via propagation of Gaussian errors).  
  
The argument *maxpaerr* specifies the maximum allowable error in the
position angle that is acceptable. The default is an infinite value.
From the standard propagation of errors, the error in the linearly
polarized position angle is determined from the Stokes Q and U images
(at each directional pixel for each frequency). If the position angle
error for any pixel exceeds the specified value, the position angle at
that pixel is omitted from the fit. The process generates an error for
the fit and this is used to compute the errors in the output images.

**NOTE**: *maxpaerr* is not used to mask pixels in the output images.

The argument *rmfg* is used to specify a foreground RM value. For
example, you may know the mean RM in some direction out of the Galaxy,
then including this can improve the algorithm by reducing ambiguity. The
parameter *rmmax* specifies the maximum absolute RM value that should be
solved for. This quite an important parameter. If you leave it at the
default, zero, no ambiguity handling will be used. So some a priori
information should be supplied; this is the basic problem with rotation
measure algorithms.



##  Calculation of Spectral Indices and Higher Order Polynomials (**spxfit**)

This application fits a power logarithmic polynomial or a logarithmic
transformed polynomial to pixel values along a specified axis of an
image or images. These functions are most often used for fitting the
spectral index and higher order terms of a spectrum. A power logarithmic
polynomial has the form

$y = \frac{c_0 x}{D^{(c_1 + c_2 \ln(x/D) + c_3 \ln(x/D)^2 + c_n \ln(x/D)^{(n - 1)})}}$

and a logarithmic transformed polynomial is simply the result of this
equation after taking the natural log of both sides so that it has the
form

$\ln(y) = c_0 + c_1 \ln(x) + c_2 \ln(x/D)^2 +  ... + c_n \ln(x/D)^n$

Because the logarithm of the ordinate values must be taken before
fitting a logarithmic transformed polynomial, all non-positive pixel
values are effectively masked for the purposes of fitting. The
coefficients of the two forms are equal to each other except that c0 in
the second equation is equal to&nbsp;$\ln(c_0)$ of the first. In the case of
fitting a spectral index, which is traditionally represented as $\alpha$, is
equal to $c_1$. In both cases, $D$ is a
normalization constant used so that abscissa values are closer to unity
when they are sent to the fitter. This generally improves the
probability that the fit will converge. This parameter may be specified
via the *div* parameter. A value of 0 (the default) indicates that the
application should determine a reasonable value for $D$, which is
determined via  
  
$D = 10^{\int(\log10(\sqrt(\min(x)*\max(x)))}$  
  
where *min(x)* and *max(x)* are the minimum and maximum abscissa values,
respectively.  
  
The inputs are:

&nbsp;\#&nbsp; spxfit :: Fit a 1-dimensional model to an image or image
region  
for determination of spectral index.  
imagename&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp; Name of the input image(s)  
box&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Rectangular box
in  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; direction coordinate blc, trc.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; Default: entire image ('').  
region&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Region of
interest. &nbsp;Default:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; Do not use a region.  
chans&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Channels to use.
Channels  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; must be contiguous. &nbsp;Default: all channels ('').  
stokes&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Stokes planes to  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; use. Planes must be contiguous. Default:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; all stokes ('').  
axis&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; The profile axis.
Default:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; use the spectral axis if one  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; exists, axis 0 otherwise (\<0).  
mask&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Mask to use.
&nbsp;Default is none.  
minpts&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Minimum number of
unmasked  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; points necessary to attempt  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; fit.  
multifit&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; If true, fit a
profile  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; along the desired axis at each  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; pixel in the specified  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; region. If false, average the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; non-fit axis pixels and do  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; a single fit to that average  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; profile. Default False.  
&nbsp;&nbsp;&nbsp;&nbsp;
spxsol&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of the
spectral index  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; function coefficient solution  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image to write.  
&nbsp;&nbsp;&nbsp;&nbsp;
spxerr&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of the
spectral index  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; function coefficient error  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image to write.  
&nbsp;&nbsp;&nbsp;&nbsp;
model&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of model  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image. Default: do not write the model  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image ('').  
&nbsp;&nbsp;&nbsp;&nbsp; residual&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of residual  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; image. Default: do not write the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; residual image ('').  
spxtype&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
'plp'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Type of
function to  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; fit. 'plp' =\> power logarithmic  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; polynomial, 'ltp' =\>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; logarithmic transformed polynomial.  
spxest&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Initial
estimates for the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; spectral index function  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; coefficients.  
spxfix&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Fix the
corresponding spectral index function  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; coefficients during the fit. True=\>hold fixed.  
div&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
0&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Divisor (numerical
value or  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; quantity) to use in the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; logarithmic terms of the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; plp or ltp function. 0 =\>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; calculate a useful value on the fly.  
wantreturn&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Should a record
summarizing  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; the results be returned?  
logresults&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output results
to logger?  
logfile&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; File in which to
log  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; results. Default is not to write a  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; logfile.  
sigma&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Standard deviation
array or image name(s).  
&nbsp;&nbsp;&nbsp;&nbsp; outsigma&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Name of output
image used  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; for standard deviation. Ignored  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; if sigma is empty.

For more than a single input image or cube, all images must have the
same dimensions along all axes other than the fit axis. *multifit* will
perform a per-pixel fit, otherwise there will be a single value over the
entire region.

&nbsp;



## Search for Spectral Line Rest Frequencies (**slsearch**)

The **slsearch** task allows the spectral line enthusiast to find their
favorite spectral lines in subset of the [Splatalogue spectral line
catalog](http://www.splatalogue.net) which is distributed with CASA. In
addition, one can export custom catalogs from Splatalogue and import
them to CASA using the task **splattotable** (next section) or tool
method **sl.splattotable**. One can even import catalogs with lines not
in Splatalogue using the same file format.  
  
The inputs to **slsearch** are as follows:

\#&nbsp; slsearch :: Search a spectral line table.  
tablename&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Input spectral
line table name to  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; search. If not specified, use the  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; default table in the system.  
outfile&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Results table
name. Blank means do not  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; write the table to disk.  
freqrange&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp; \[84, 90\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp; Frequency range in GHz.  
species&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[''\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Species to
search for.  
reconly&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
False&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; List only NRAO
recommended  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; frequencies.  
chemnames&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[''\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Chemical names
to search for.  
qns&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[''\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Resolved
quantum numbers to search  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; for.  
rrlinclude&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Include RRLs in
the result set?  
rrlonly&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
False&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Include only
RRLs in the result set?  
&nbsp;&nbsp;&nbsp;&nbsp; intensity&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; CDMS/JPL intensity
range. -1 -\> do not  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; use an intensity range.  
&nbsp;&nbsp;&nbsp;&nbsp;
smu2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; S\*mu\*mu range in
Debye\*\*2. -1 -\> do  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; not use an S\*mu\*mu range.  
&nbsp;&nbsp;&nbsp;&nbsp;
loga&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; log(A) (Einstein
coefficient) range.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; -1 -\> do not use a loga range.  
&nbsp;&nbsp;&nbsp;&nbsp;
eu&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Upper energy state
range in Kelvin. -1  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; -\> do not use an eu range.  
&nbsp;&nbsp;&nbsp;&nbsp;
el&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
-1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Lower energy state
range in Kelvin. -1  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; -\> do not use an el range.  
  
verbose&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; List result set
to logger (and  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; optionally logfile)?  
&nbsp;&nbsp;&nbsp;&nbsp;
logfile&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; List result set to
this logfile (only  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; used if verbose=True).  
&nbsp;&nbsp;&nbsp;&nbsp;
append&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; If true, append
to logfile if it  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; already exists, if false overwrite  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; logfile if it exists. Only used if  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; verbose=True and logfile not blank.  
  
wantreturn&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; If true, return
the spectralline tool  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\#&nbsp;&nbsp; associated with the result set.

  
The table is provided in the *tablename* parameter but if it is blank
(the default), the catalog which is included with CASA will be used.
Searches can be made in a parameter space with large dimensionality:

&nbsp; &nbsp;
*freqrange*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Frequency range in GHz.  
&nbsp;&nbsp;&nbsp;
*species*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Species to search for.  
&nbsp;&nbsp;&nbsp;
*reconly*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
List only NRAO recommended frequencies.  
&nbsp;&nbsp;&nbsp;
*chemnames*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Chemical
names to search for.  
&nbsp;&nbsp;&nbsp;
*qns*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Resolved quantum numbers to search for.  
&nbsp;&nbsp;&nbsp; *intensity* &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;
&nbsp;&nbsp;&nbsp; CDMS/JPL intensity range.  
&nbsp;&nbsp;&nbsp;
*smu2*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$S\mu^{2}$
range in Debye$^2$.  
&nbsp;&nbsp;&nbsp;
*loga*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
log(A) (Einstein coefficient) range.  
&nbsp;&nbsp;&nbsp;
*el*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Lower energy state range in Kelvin.  
&nbsp;&nbsp;&nbsp;
*eu*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Upper energy state range in Kelvin.  
&nbsp;&nbsp;&nbsp;
*rrlinclude*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Include RRLs in the result set?  
&nbsp;&nbsp;&nbsp;
*rrlonly*&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Include only RRLs in the result set?

Notation is as found in the Splatalogue catalog.  
  
Example:  
Search for all lines of the species HOCN and HOCO$^+$ in the
200-300GHz range:

slsearch(outfile='myresults.tbl', freqrange = \[200,300\],  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; species=\['HOCN',
'HOCO+'\])

The task can also return a python dictionary if assigned a variable
like:

myLines = slsearch(outfile='myresults.tbl', freqrange = \[200,300\],  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; species=\['HOCN',
'HOCO+'\])



##  Convert Exported Splatalogue Catalogs to CASA Tables (**splattotable**)

In some cases, the internal spectral line catalog may not contain the
lines in which one is interested. In that case, one can export a catalog
from [Splatalogue](http://www.splatalogue.net) or even create their own
'by hand' (be careful to get the format exactly right though!).
CASA&rsquo;s task **splattotable** can then be used to create a CASA
table that contains these lines and can be searched:

\#&nbsp; splattotable :: Convert a downloaded Splatalogue spectral line
list to a casa table.  
filenames&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\[''\]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Files
containing Splatalogue lists.  
table&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
''&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Output table
name.  
wantreturn&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
=&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
True&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \#&nbsp; Do you want the
task to return a spectralline tool attached to the results table?

A search in Splatalogue will return a catalog that can be saved in a
file (look for the 'Export' section after the results on the search
results page). The exported filename(s) should be entered in the
*filenames* parameter of **splattotable**. The downloaded files must be
in a specific format for this task to succeed. If you use the
Splatalogue '*Export CASA fields*' feature, you should have no
difficulties.

&nbsp;

&nbsp;

Bibliography

  
<sup>1.\ Leahy,\ J.\~P.,\ Pooley,\ G.\~G.,\ &\ Jagers,\ W.\~J.\ 1986,\ A&A,\ 156,\ 234\ ([ADS](http://adsabs.harvard.edu/abs/1986A%26A...156..234L))\ [&larrhk;](#ref-cit1 "Jump back to citation 1 in the text.")</sup>